This notebook creates a 2D sagittal slice. The slice is transposed to be in x-y plane

In [ ]:
import nibabel as nib
import h5py
import numpy as np
import scipy.ndimage

In [ ]:
filen='sample.h5' #output filename
zoom = 4 #zoom factor for increasing number of spins
resolution= 2/zoom #sample resultion in mm, assuming same resolution in all dimensions

In [ ]:
#coordinates for the slice
x = 82
xi = 83
y = 60
yi = 190
z = 85
zi = 245

In [ ]:
#load phantom files
shape = (281,256,256)
pd = np.fromfile("Phantom/proton_density_act_1.bin", dtype=np.float32).reshape(shape)
t1 = np.fromfile("Phantom/t1_act_1.bin", dtype=np.float32).reshape(shape)
t2 = np.fromfile("Phantom/t2_act_1.bin", dtype=np.float32).reshape(shape)

In [ ]:
#extract slice
pd = pd[z:zi,y:yi,x][:,::-1]
t1 = t1[z:zi,y:yi,x][:,::-1]
t2 = t2[z:zi,y:yi,x][:,::-1]

In [ ]:
#zoom to increase number of spins
resized_pd = scipy.ndimage.zoom(pd, zoom, order=0)
resized_t1 = scipy.ndimage.zoom(t1, zoom, order=0)
resized_t2 = scipy.ndimage.zoom(t2, zoom, order=0)

In [ ]:
jemris_params = np.zeros((resized_pd.shape[0], resized_pd.shape[1], 5))
jemris_params[:, :, 0] = resized_pd #proton density
jemris_params[:, :, 1] = np.reciprocal(resized_t1, where=resized_t1>0) #T1
jemris_params[:, :, 2] = np.reciprocal(resized_t2, where=resized_t2>0) #T2
jemris_params[:, :, 3] = jemris_params[:, :, 2] # T2* set as T2
# chemical shift values left as 0

In [ ]:
#write file
sample_file = h5py.File(filen, "w")
sample_file.create_dataset("sample/data", data=jemris_params)
sample_file.create_dataset("sample/resolution", data=np.array([resolution, resolution, resolution]))
sample_file.create_dataset("sample/offset", (3,)) # empty tuple, patient is centred
sample_file.close()

Motion model

In [ ]:
trace_file='Breathing_traces/cos_trace_shifted.txt' #surrogate trace filename
res=100 #distance between timepoints in ms
out_file='motion_model_cos_shifted.h5' #output filename

In [ ]:
trace = np.loadtxt(trace_file)

In [ ]:
ap_all=nib.load("Phantom/modelComp_ap.nii.gz").get_fdata() #load AP component file
ap_all=ap_all[x:xi,y:yi,z:zi,0,:] #extract slice
ap_all=np.transpose(ap_all, (0, 2, 1, 3)) #transpose the slice to x-y plane
ap = np.zeros((1,160,130,3))
ap[0,:,:,0]=ap_all[0,:,::-1,1] #reorder components since we are changing plane
ap[0,:,:,1]=ap_all[0,:,::-1,2]
ap[0,:,:,2]=ap_all[0,:,::-1,0]

In [ ]:
si_all=nib.load("Phantom/modelComp_si.nii.gz").get_fdata()
si_all=si_all[x:xi,y:yi,z:zi,0,:]
si_all = np.transpose(si_all, (0, 2, 1, 3))
si = np.zeros((1,160,130,3))
si[0,:,:,0]=si_all[0,:,::-1,1]
si[0,:,:,1]=si_all[0,:,::-1,2]
si[0,:,:,2]=si_all[0,:,::-1,0]

In [ ]:
offset_all=nib.load("Phantom/modelComp_offset.nii.gz").get_fdata()
offset_all=offset_all[x:xi,y:yi,z:zi,0,:]
offset_all = np.transpose(offset_all, (0, 2, 1, 3))
offset = np.zeros((1,160,130,3))
offset[0,:,:,0]=offset_all[0,:,::-1,1]
offset[0,:,:,1]=offset_all[0,:,::-1,2]
offset[0,:,:,2]=offset_all[0,:,::-1,0]

In [ ]:
#write file
with h5py.File(out_file, 'w') as motion:
    motion_resolution = motion.create_dataset("model/resolution", data=np.array([res, 2, 2, 2]))
    motion_offset = motion.create_dataset("model/offset", data=np.array([0, 0, 0]))
    motion_ap = motion.create_dataset("model/ap", data=ap)
    motion_si = motion.create_dataset("model/si", data=si)
    motion_model_offset = motion.create_dataset("model/model_offset", data=offset)
    motion_trace = motion.create_dataset("model/breathing_trace", data=trace)